In [1]:
# cleanup of bad session
import pyvisa
rm = pyvisa.ResourceManager()
for res in rm.list_resources():
    try:
        inst = rm.open_resource(res)
        inst.write("*RST;*CLS")      # SCPI reset + clear status
        inst.write("smua.reset()")   # TSP reset
        inst.write("smua.abort()")   # in case it was triggering
        inst.write("smua.output = smua.OUTPUT_OFF")
        inst.close()
    except Exception:
        pass

In [2]:
rm.open_resource('USB0::0x05E6::0x2601::4120420::INSTR').query("*IDN?").strip()

'Keithley Instruments Inc., Model 2601B, 4120420, 3.2.2'

In [33]:
import pyvisa
from keithley2600 import Keithley2600
# 1) List available VISA resources
rm = pyvisa.ResourceManager('@ivi') 
resources = rm.list_resources()
print("Available VISA resources:", resources)

# 2) Find the 2601B by querying *IDN?
keithley_addr = None
for addr in resources:
    try:
        temp = rm.open_resource(addr)
        idn = temp.query('*IDN?').strip()
        temp.close()
        if '2601' in idn and 'Keithley' in idn:
            keithley_addr = addr
            print(f"Found Keithley 2601B at {addr}: {idn}")
            break
    except Exception:
        pass

if not keithley_addr:
    raise ValueError("Keithley 2601B not found. Check your USB/VISA setup.")

# 3) Hand off to the keithley2600 driver
keithley = Keithley2600(keithley_addr, 
                            visa_library='C:/Windows/System32/visa64.dll',
                            raise_keithley_errors=True)
keithley.connect()
idn = keithley.connection.query('*IDN?').strip()
print("Driver connected to:", idn)
print("Keithey connected:", keithley.connected)

Available VISA resources: ('USB0::0x05E6::0x2601::4120420::INSTR', 'ASRL12::INSTR')
Found Keithley 2601B at USB0::0x05E6::0x2601::4120420::INSTR: Keithley Instruments Inc., Model 2601B, 4120420, 3.2.2
Driver connected to: Keithley Instruments Inc., Model 2601B, 4120420, 3.2.2
Keithey connected: True


In [35]:
keithley.reset()

In [30]:
keithley.disconnect()
print("Keithley disconnected:", not keithley.connected)

Keithley disconnected: True


In [ ]:
import time
import csv
from datetime import datetime, timezone
from keithley2600 import Keithley2600

# ── Parameters ─────────────────────────────────────────────────────────
RESOURCE   = "USB0::0x05E6::0x2601::4120420::INSTR"
TARGET_HZ  = 100              # your measured sustainable rate
PERIOD     = 1.0 / TARGET_HZ  # ≈ 2.22 ms
DURATION   = 2.0              # seconds to run
VOLTAGE    = 10.0             # volts to source
N          = 300
# ──────────────────────────────────────────────────────────────────────

# 1) Connect & configure
k   = Keithley2600(RESOURCE, visa_library="@ivi")
k.connect()
smu = k.smua
smu.reset()
k.set_integration_time(smu, 0.05)
k.apply_voltage(smu, VOLTAGE)

# 2) Measurement loop
results = []
t_loop = time.perf_counter()
for i in range(N):
    t0 = time.perf_counter()
    ts = datetime.now(timezone.utc).isoformat()
    curr = smu.measure.i()
    results.append((ts, VOLTAGE, curr))
    
    dt = time.perf_counter() - t0

total = time.perf_counter() - t_loop
print(f"Captured {len(results)} points in {total:.3f}s ⇒ {len(results)/total:.1f} Hz")

# 3) Save CSV
with open("data_450Hz.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["time_utc","voltage_V","current_A"])
    w.writerows(results)
print("Saved → data_450Hz.csv")

# 4) Cleanup
smu.output = smu.OUTPUT_OFF
k.disconnect()
print("Output OFF, session closed.")


Captured 200 points in 10.539s ⇒ 19.0 Hz
Saved → data_450Hz.csv
Output OFF, session closed.


In [35]:
import time
import csv
from datetime import datetime, timezone
import plotly.express as px
from keithley2600 import Keithley2600

# ── Parameters ─────────────────────────────────────────────────────────────
RESOURCE      = "USB0::0x05E6::0x2601::4120420::INSTR"
DURATION_S    = 40.0              # total capture time in seconds
SAVE_INTERVAL = 10.0              # seconds between file saves
SOURCE_VOLT   = 10.0              # volts to source
# ────────────────────────────────────────────────────────────────────────────

# Connect to the instrument
k   = Keithley2600(RESOURCE, visa_library='@ivi')
k.connect()
smu = k.smua

# Configure SMU A for DC sourcing and fastest integration
smu.reset()
k.set_integration_time(smu, 0.001)
k.apply_voltage(smu, SOURCE_VOLT)

# Prepare for acquisition
filename   = 'data_periodic.csv'
deltas     = []
buffer_rows = []

start_time = time.perf_counter()
last_save  = start_time
prev_time  = start_time

# Open CSV for periodic writing
with open(filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['time_utc', 'voltage_V', 'current_A', 'delta_s'])
    
    # Acquisition loop
    while True:
        now = time.perf_counter()
        elapsed = now - start_time
        if elapsed >= DURATION_S:
            break
        
        # Timestamp and measurement
        ts      = datetime.now(timezone.utc).isoformat()
        current = smu.measure.i()
        
        # Compute inter-sample interval
        delta = now - prev_time
        prev_time = now
        
        # Store row
        buffer_rows.append((ts, SOURCE_VOLT, current, delta))
        deltas.append(delta)
        
        # Periodic save
        if now - last_save >= SAVE_INTERVAL:
            writer.writerows(buffer_rows)
            f.flush()
            buffer_rows.clear()
            last_save = now
    
    # Write any remaining rows
    if buffer_rows:
        writer.writerows(buffer_rows)
        f.flush()

print(f"Recorded {len(deltas)} points over {DURATION_S} seconds into '{filename}'")

# Cleanup
smu.output = smu.OUTPUT_OFF
k.disconnect()
print("Output turned OFF and session closed.")


Recorded 16985 points over 40.0 seconds into 'data_periodic.csv'
Output turned OFF and session closed.
